In [7]:
import cv2
import numpy as np
import time

In [8]:
# 初始化摄像头
cap = cv2.VideoCapture(4)

In [9]:
# 设置摄像头的帧大小
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1080)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 980)

True

In [10]:
lower_blue = np.array([100, 50, 50])
upper_blue = np.array([130, 255, 255])
lower_red = np.array([0, 50, 50])
upper_red = np.array([10, 255, 255])

In [12]:
start_time = time.time()
while True:
    # 读取一帧图像
    ret, frame = cap.read()

    if not ret:
        break

    # 转换图像到HSV色彩空间
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # 使用颜色阈值找到蓝色目标
    blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)
    _, blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#这_,把两输入改成单输入

     # 寻找蓝色目标的区域
    for contour in blue_contours:
        if cv2.contourArea(contour) > 1000:
            # 找到目标的上下两个点
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = np.int0(box)

            # 寻找上下两个点
            top_point = tuple(box[box[:, 1].argmin()])
            bottom_point = tuple(box[box[:, 1].argmax()])

            # 在图像上标记上下两个点的位置
            cv2.circle(frame, top_point, 5, (0, 255, 0), -1)
            cv2.circle(frame, bottom_point, 5, (0, 255, 0), -1)

            # 连接上下两点并标注交点
            mid_point = ((top_point[0] + bottom_point[0]) // 2, (top_point[1] + bottom_point[1]) // 2)

            cv2.line(frame, top_point, bottom_point, (0, 255, 0), 2)
            cv2.circle(frame, mid_point, 5, (0, 0, 255), -1)

    # 在图像的左上角显示当前的帧率
    cv2.putText(frame, f"FPS: {int(1 / (time.time() - start_time))}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                (0, 255, 0), 2)

    # 显示图像
    cv2.imshow('Frame', frame)

    # 按 'q' 键退出循环
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放摄像头并关闭窗口
cap.release()
cv2.destroyAllWindows()